In [30]:
import os
os.chdir(r"c:\Users\britt\Desktop\YH\Applicerad AI\job_discrimination_sandbox")

import gensim
import numpy as np
import pandas as pd
import pickle
from catboost import CatBoostRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [4]:
google_model = gensim.models.KeyedVectors.load_word2vec_format("c:/Users/britt/Downloads/GoogleNews-vectors-negative300.bin.gz", binary=True)

In [23]:
class_df = pd.read_csv("data/cleaned_data/bulletins_w_labels_and_content.csv", dtype={'ID': object})

In [35]:
X = class_df["Cleaned text"]

tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X).toarray()
y_classes = class_df["Numeric label 70/30"]

In [36]:
with open("data/vectorizers/tfidf.pkl", "wb") as write_file:
    pickle.dump(tfidf_vectorizer, write_file)

In [15]:
clf = LogisticRegression(C=2.672, class_weight="balanced")
log_reg_tfidf_model = clf.fit(X_tfidf, y_classes)

In [31]:
with open("data/models/log_reg_tfidf.pkl", "wb") as write_file:
    pickle.dump(log_reg_tfidf_model, write_file)

In [34]:
with open("data/vectorizers/tfidf.pkl", "wb") as write_file:
    pickle.dump(X_tfidf, write_file)

In [17]:
corpus = list(class_df["Cleaned text"])

tfidf_vectorizer2 = TfidfVectorizer()
tfidf_vectorizer2.fit_transform(corpus)

<177x3835 sparse matrix of type '<class 'numpy.float64'>'
	with 68003 stored elements in Compressed Sparse Row format>

In [20]:
vocabulary = tfidf_vectorizer2.get_feature_names_out()
documents_embeddings = []
documents_scaled_embeddings = []
for doc in corpus:
    word_embeddings = []
    scaled_embeddings  = []
    doc_list = doc.split()
    for word in doc_list:
        if word in google_model.key_to_index.keys():
            embedding = google_model[word]
            word_embeddings.append(embedding)
            index = np.where(vocabulary == word)[0]
            try:
                scaled_embeddings.append(embedding * tfidf_vectorizer2.idf_[index])
            except ValueError:
                pass
    documents_embeddings.append(word_embeddings)
    documents_scaled_embeddings.append(scaled_embeddings)

In [21]:
scaled_doc_vectors = [np.average(doc, axis=0) for doc in documents_scaled_embeddings]
X_scaled = np.array(scaled_doc_vectors)

In [22]:
clf = LogisticRegression(C=6.98)
log_reg_scaled_emb_model = clf.fit(X_scaled, y_classes)

In [32]:
with open("data/models/log_reg_scaled_emb.pkl", "wb") as write_file:
    pickle.dump(log_reg_scaled_emb_model, write_file)

In [25]:
distribution_df = pd.read_csv("data/cleaned_data/bulletins_labels_share_content.csv", dtype={'ID': object})

In [27]:
y_distributions = list(zip(list(distribution_df["Male share (unknown gender included)"]), list(distribution_df["Female share (unknown gender included)"])))

In [37]:
param = {}
param['learning_rate'] = 0.018
param['depth'] = 12
param['l2_leaf_reg'] = 1.5
param['min_child_samples'] = 32
param['grow_policy'] = 'Depthwise'
param['eval_metric'] = 'MultiRMSE'
param['loss_function'] = 'MultiRMSE'
param['od_type'] = 'iter'
param['od_wait'] = 20
param['random_state'] = 1
param['logging_level'] = 'Silent'


regressor = CatBoostRegressor(**param)

cat_boost_regr_model = regressor.fit(X_scaled, y_distributions, early_stopping_rounds=100)

In [40]:
with open("data/models/cat_boost_regr.pkl", "wb") as write_file:
    pickle.dump(cat_boost_regr_model, write_file)

In [39]:
cat_boost_regr_model